In [1]:
import os
import fnmatch

import loadData as ld

import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer

# calculate accuracy of class predictions
from sklearn import metrics

from sklearn.model_selection import train_test_split

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized


In [3]:
topicNumber = -1
lastTopic = ""

data = []

x_test = []
y_test = []

for dirpath, dirs, files in os.walk('../data/bbc'):
    for filename in fnmatch.filter(files, '*.txt'):
        if(lastTopic != dirpath):
            topicNumber += 1
            lastTopic = dirpath
            print("Topic : ", lastTopic, " associé à : ", topicNumber)
            
        with open(os.path.join(dirpath, filename), encoding="ISO-8859-1") as f:
            data.append((clean(f.read()), topicNumber))
            
data_train, data_test =  train_test_split(data, train_size=0.95)

#Unzip the list of tuple into two list
x_train, y_train = zip(*data_train)
x_test, y_test = zip(*data_test)


Topic :  ../data/bbc/sport  associé à :  0
Topic :  ../data/bbc/tech  associé à :  1
Topic :  ../data/bbc/business  associé à :  2
Topic :  ../data/bbc/entertainment  associé à :  3
Topic :  ../data/bbc/politics  associé à :  4


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=4)
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)

x_test_tfidf = tfidf_vectorizer.transform(x_test)

In [5]:
#prediction
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(x_train_tfidf, y_train)

In [6]:
predicted = clf.predict(x_test_tfidf)

print(metrics.accuracy_score(y_test, predicted))

1.0


In [7]:
x_test_tfidf.shape

(112, 9829)

In [9]:
#Sort CNN data into topics 
import tarfile
import shutil
import collections

directory = "../data/cnn/stories"
stat = []
    
for name in os.listdir(directory):
        filename = directory + '/' + name
        # load document
        doc = ld.load_doc(filename)
        
        # split into story and highlights
        story, _ = ld.split_story(doc)
        tfidf_cnn = tfidf_vectorizer.transform([clean(story)])
        predicted_cnn = clf.predict(tfidf_cnn)
        
        if(predicted_cnn[0] == 0):
            shutil.copy2(filename, "../data/classification/sport")
        elif(predicted_cnn[0] == 1):
            shutil.copy2(filename, "../data/classification/tech")
        elif(predicted_cnn[0] == 2):
            shutil.copy2(filename, "../data/classification/business")
        elif(predicted_cnn[0] == 3):
            shutil.copy2(filename, "../data/classification/entertainment")
        else:
            shutil.copy2(filename, "../data/classification/politics")
            
        stat.append(predicted_cnn[0])
            
collections.Counter(stat)


Counter({0: 13420, 1: 9907, 2: 19690, 3: 12885, 4: 36677})

In [ ]:
story